In [1]:
import pandas as pd 
import numpy as np
import warnings

## 데이터 불러오기

In [2]:
train = pd.read_csv('./Sample_Dataset/train.csv', encoding='UTF-8')
cust_tr = pd.read_csv('./Sample_Dataset/cust_tr.csv', encoding='UTF-8')
test = pd.read_csv('./Sample_Dataset/test.csv', encoding='UTF-8')

C:\Users\user\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.head()

,CLNT_ID,SESS_ID,HITS_SEQ,PD_C,PD_ADD_NM,PD_BRA_NM,PD_BUY_AM,PD_BUY_CT,SESS_SEQ,SESS_DT,...,TOT_SESS_HR_V,DVC_CTG_NM,ZON_NM,CITY_NM,KWD_NM,SEARCH_CNT,PD_NM,CLAC1_NM,CLAC2_NM,CLAC3_NM
0,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,남아 파자마,5,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
1,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,주니어 파자마,4,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
2,200251,8343839,211,643942,사이즈:75 / 1개,오르시떼,"28,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,파자마,2,남아동 코튼륀 긴소매 상하(55-75) OR17FBGE028055 - 60,속옷/양말/홈웨어,유아동속옷,유아동내의
3,200251,8343839,211,537989,사이즈:85 / 1개,오르시떼,"33,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,남아 파자마,5,여아동 셔틀콕 긴소매 상하(80-85) OR18SGGA0040802 - 80,속옷/양말/홈웨어,유아동속옷,유아동내의
4,200251,8343839,211,537989,사이즈:85 / 1개,오르시떼,"33,000",1,30,20180514,...,"2,262",mobile,Gyeonggi-do,Seongnam-si,주니어 파자마,4,여아동 셔틀콕 긴소매 상하(80-85) OR18SGGA0040802 - 80,속옷/양말/홈웨어,유아동속옷,유아동내의


In [4]:
cust_tr.head()

,CLNT_ID,LABEL
0,200251,F
1,183658,F
2,66492,F
3,262675,F
4,37314,F


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2393262 entries, 0 to 2393261
Data columns (total 21 columns):
 #   Column           Dtype  
---  ------           -----  
 0   CLNT_ID          int64  
 1   SESS_ID          int64  
 2   HITS_SEQ         int64  
 3   PD_C             int64  
 4   PD_ADD_NM        object 
 5   PD_BRA_NM        object 
 6   PD_BUY_AM        object 
 7   PD_BUY_CT        int64  
 8   SESS_SEQ         int64  
 9   SESS_DT          int64  
 10  TOT_PAG_VIEW_CT  float64
 11  TOT_SESS_HR_V    object 
 12  DVC_CTG_NM       object 
 13  ZON_NM           object 
 14  CITY_NM          object 
 15  KWD_NM           object 
 16  SEARCH_CNT       int64  
 17  PD_NM            object 
 18  CLAC1_NM         object 
 19  CLAC2_NM         object 
 20  CLAC3_NM         object 
dtypes: float64(1), int64(8), object(12)
memory usage: 383.4+ MB


## 데이터 전처리

In [6]:
train['PD_BUY_AM']  = train['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
train['PD_BUY_CT']  = train['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_AM']  = test['PD_BUY_AM'].map(lambda x: int(str(x).replace(',','')) )
test['PD_BUY_CT']  = test['PD_BUY_CT'].map(lambda x: int(str(x).replace(',','')) )

## feature 생성

In [7]:
features = []
features_te = []

**[총구매액, 구매건수, 평균구매액, 최대구매액]**

In [8]:
train['AMOUNT'] = train['PD_BUY_AM'] * train['PD_BUY_CT']
test['AMOUNT'] = test['PD_BUY_AM'] * test['PD_BUY_CT']

In [9]:
f = train.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features.append(f);display(f)

f_te = test.groupby('CLNT_ID')['AMOUNT'].agg([('총구매액', np.sum),
                                            ('구매건수', np.size),
                                            ('평균구매액', lambda x : np.round(np.mean(x))),
                                            ('최대구매액', np.max)]).reset_index()
features_te.append(f_te) ; display(f_te)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,1,1276000,16,79750,99000
1,4,125200,6,20867,37000
2,5,380000,12,31667,42000
3,6,148900,7,21271,44900
4,7,457900,16,28619,40000
...,...,...,...,...,...
184167,263099,973000,112,8688,22000
184168,263100,22400,2,11200,12900
184169,263101,135600,4,33900,33900
184170,263102,59400,15,3960,9720


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액
0,0,86500,2,43250,81000
1,2,560000,11,50909,62100
2,3,851200,8,106400,110400
3,16,9500,1,9500,9500
4,17,5800,2,2900,2900
...,...,...,...,...,...
78927,263083,117300,3,39100,39100
78928,263085,1562720,39,40070,74500
78929,263086,1836300,44,41734,89000
78930,263094,10000,1,10000,10000


**[주말방문비율]**

In [10]:
train['date'] = pd.to_datetime(train['SESS_DT'], format= '%Y%m%d')
test['date'] = pd.to_datetime(test['SESS_DT'], format= '%Y%m%d')

In [11]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features.append(f); display(f)

f_te = test.groupby('CLNT_ID')['date'].agg([
    ('주말방문비율', lambda x: np.mean(x.dt.dayofweek>4))]).reset_index()
features_te.append(f_te);display(f_te)

,CLNT_ID,주말방문비율
0,1,0.125000
1,4,0.333333
2,5,0.000000
3,6,0.857143
4,7,0.312500
...,...,...
184167,263099,0.000000
184168,263100,0.500000
184169,263101,0.000000
184170,263102,0.000000


,CLNT_ID,주말방문비율
0,0,1.000000
1,2,0.000000
2,3,1.000000
3,16,1.000000
4,17,1.000000
...,...,...
78927,263083,0.000000
78928,263085,0.333333
78929,263086,0.000000
78930,263094,0.000000


**[계절방문비율]**

In [12]:
f = train.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features.append(f); f

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,1,0.375000,0.625000,0.0,0.0
1,4,0.666667,0.333333,0.0,0.0
2,5,0.000000,0.000000,1.0,0.0
3,6,1.000000,0.000000,0.0,0.0
4,7,0.437500,0.562500,0.0,0.0
...,...,...,...,...,...
184167,263099,0.000000,1.000000,0.0,0.0
184168,263100,0.500000,0.500000,0.0,0.0
184169,263101,1.000000,0.000000,0.0,0.0
184170,263102,1.000000,0.000000,0.0,0.0


In [13]:
f_te = test.groupby('CLNT_ID')['date'].agg([
    ('봄-구매비율', lambda x: np.mean( x.dt.month.isin([3,4,5]))),
    ('여름-구매비율', lambda x: np.mean( x.dt.month.isin([6,7,8]))),
    ('가을-구매비율', lambda x: np.mean(x.dt.month.isin([9,10,11]))),
    ('겨울-구매비율', lambda x: np.mean( x.dt.month.isin([1,2,12])))
]).reset_index()
features_te.append(f_te); f_te

,CLNT_ID,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,0,0.000000,1.000000,0.000000,0.0
1,2,0.000000,0.909091,0.090909,0.0
2,3,0.000000,0.000000,1.000000,0.0
3,16,0.000000,0.000000,1.000000,0.0
4,17,0.000000,1.000000,0.000000,0.0
...,...,...,...,...,...
78927,263083,0.000000,1.000000,0.000000,0.0
78928,263085,0.461538,0.538462,0.000000,0.0
78929,263086,0.227273,0.772727,0.000000,0.0
78930,263094,1.000000,0.000000,0.000000,0.0


## data 생성

In [14]:
X_train = pd.DataFrame({'CLNT_ID': train.CLNT_ID.unique()})
for f in features :
    X_train = pd.merge(X_train, f, how='left')
    
X_train = X_train.fillna(0)

In [15]:
X_test = pd.DataFrame({'CLNT_ID': test.CLNT_ID.unique()})
for f in features_te :
    X_test = pd.merge(X_test, f, how='left')
    
X_test = X_test.fillna(0)

In [16]:
display(X_train)
display(X_test)

,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,200251,183000,6,30500,33000,0.000000,1.000000,0.000000,0.0,0.0
1,183658,30200,2,15100,23900,0.000000,0.000000,0.000000,1.0,0.0
2,66492,539500,8,67438,90850,1.000000,0.000000,1.000000,0.0,0.0
3,262675,344500,9,38278,59000,0.666667,0.222222,0.777778,0.0,0.0
4,37314,253500,10,25350,47600,0.000000,0.000000,0.000000,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
184167,84857,136200,3,45400,53100,0.000000,1.000000,0.000000,0.0,0.0
184168,159461,246700,6,41117,74300,0.333333,0.000000,1.000000,0.0,0.0
184169,37310,90000,3,30000,30000,0.000000,0.000000,1.000000,0.0,0.0
184170,77442,1402340,53,26459,92000,0.358491,0.471698,0.528302,0.0,0.0


,CLNT_ID,총구매액,구매건수,평균구매액,최대구매액,주말방문비율,봄-구매비율,여름-구매비율,가을-구매비율,겨울-구매비율
0,153000,836350,12,69696,85800,0.166667,0.166667,0.833333,0.0,0.0
1,219727,205000,6,34167,52000,0.666667,0.000000,1.000000,0.0,0.0
2,32605,588000,1,588000,588000,0.000000,1.000000,0.000000,0.0,0.0
3,262090,198600,4,49650,63600,1.000000,1.000000,0.000000,0.0,0.0
4,55484,61400,3,20467,22000,0.333333,0.000000,1.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
78927,46391,85000,1,85000,85000,0.000000,0.000000,1.000000,0.0,0.0
78928,251340,606900,9,67433,83300,1.000000,0.000000,1.000000,0.0,0.0
78929,149916,556500,18,30917,39000,1.000000,0.000000,0.000000,1.0,0.0
78930,234326,15000,1,15000,15000,0.000000,0.000000,0.000000,1.0,0.0


In [17]:
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [18]:
X_train = X_train.drop(columns=['CLNT_ID'])
X_test = X_test.drop(columns=['CLNT_ID'])

In [19]:
y_train = cust_tr.LABEL

## 모델 학습

**[데이터 분리]**

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

**[DecisionTree]**

In [21]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=0)

In [22]:
dt.fit(X_train,y_train)

DecisionTreeClassifier(random_state=0)

In [23]:
print('모델 학습 점수 :',dt.score(X_train,y_train))

모델 학습 점수 : 0.9667313062364257


In [24]:
from sklearn.model_selection import cross_val_score

In [25]:
scores1 = cross_val_score(dt, X_train, y_train, cv=5)

print("교차 검증 점수: {}".format(scores1))

교차 검증 점수: [0.77710984 0.78056159 0.77183525 0.77753646 0.7788551 ]


In [26]:
print("교차 검증 평균 점수: {:.2f}".format(scores1.mean()))

교차 검증 평균 점수: 0.78


In [27]:
print('테스트 점수 :',dt.score(X_val,y_val))

테스트 점수 : 0.7795373923115906


In [28]:
#참고 코드
#scores = cross_val_score(knn, X_train, y_train, scoring='neg_log_loss', cv=5)

**[KNeighbors]**

In [29]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

In [30]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=3)

In [31]:
print('모델 학습 점수 :',knn.score(X_train,y_train))

모델 학습 점수 : 0.8750465404902265


In [32]:
scores2 = cross_val_score(knn, X_train, y_train, cv=5)

print("교차 검증 점수: {}".format(scores2))

교차 검증 점수: [0.82655911 0.82636519 0.82485262 0.82485262 0.822991  ]


In [33]:
print("교차 검증 평균 점수: {:.2f}".format(scores2.mean()))

교차 검증 평균 점수: 0.83


In [34]:
print('테스트 점수 :',knn.score(X_val,y_val))

테스트 점수 : 0.8267574024469703


**cv를 하이퍼파라미터 튜닝에 적용하기**

In [35]:
print('테스트 점수 :',dt.score(X_val,y_val))

테스트 점수 : 0.7795373923115906


In [36]:
from sklearn.model_selection import GridSearchCV

model_dt = DecisionTreeClassifier(random_state = 0)

param_grid = {'criterion' :['gini','entropy'],
              'max_depth':[4,5,6,8,10],
              'min_samples_leaf':[3,5,10,50,100]}
# 2 * 5 * 5 = 50 번 반복

gcv = GridSearchCV(model_dt, param_grid=param_grid, cv=3, scoring='accuracy', n_jobs=-1)
gcv.fit(X_train,y_train)

print('final params', gcv.best_params_)
print('best score', gcv.best_score_)
print(gcv.score(X_val,y_val))

final params {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 3}
best score 0.8634967423297583
0.8642040107145442


In [37]:
from sklearn.model_selection import StratifiedKFold

In [38]:
skf = StratifiedKFold(n_splits=4 , shuffle=False)

In [39]:
from sklearn.model_selection import GridSearchCV

model_dt = DecisionTreeClassifier(random_state = 0)

param_grid = {'criterion' :['gini','entropy'],
              'max_depth':[4,5,6,8,10],
              'min_samples_leaf':[3,5,10,50,100]}
# 2 * 5 * 5 = 50 번 반복

gcv = GridSearchCV(model_dt, param_grid=param_grid, cv=skf, scoring='accuracy', n_jobs=-1)
gcv.fit(X_train,y_train)

print('final params', gcv.best_params_)
print('best score', gcv.best_score_)
print(gcv.score(X_val,y_val))

final params {'criterion': 'entropy', 'max_depth': 4, 'min_samples_leaf': 5}
best score 0.8635122556624264
0.8641678129298487


## The End